In [ ]:
pip install ekphrasis

In [ ]:
pip install sastrawi

In [1]:
import re
import string
import pandas as pd
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [2]:
df = pd.read_excel('D:\\tweets_dataframe_cleaned.xlsx')

In [3]:
df

,userTags,timeStamps,tweets,replies,retweets,likes,views
0,@ShenoWirang,2024-01-10T07:53:20.000Z,"Ketika Pilpres Filipina tahun 2022 ""Bongbong"" ...",11.0,9.0,58,2 rb
1,@detikcom,2024-01-07T16:02:06.000Z,"#DebatPilpres Dalam menutup debat, Anies-Cak I...",8.0,15.0,90,17 rb
2,@P_kenjeran,2024-01-09T03:09:30.000Z,Saya rasa Prabowo pas sekolah lewat jalur khus...,1.0,4.0,10,621
3,@KompasTV,2024-01-12T12:15:00.000Z,Debat Pilpres ketiga antar capres berlangsung ...,1.0,NaN,1,1 rb
4,@kompascom,2024-01-09T22:32:13.000Z,Seorang kepala negara sekaligus kepala pemerin...,108.0,39.0,149,27 rb
5,@NasDem,2024-01-07T13:46:16.000Z,"Calon presiden (capres) nomor urut 1, Anies Ba...",3.0,65.0,213,2 rb
6,@islah_bahrawi,2024-01-08T07:59:48.000Z,"""Prabowo memiliki hak tanah 220 ribu hektar di...",209.0,877.0,3 rb,148 rb
7,@detikcom,2024-01-09T14:50:00.000Z,Sandiaga singgung soal format debat yang menya...,54.0,11.0,47,45 rb
8,@tempodotco,2024-01-10T22:04:05.000Z,Hadapi Cak Imin dan Mahfud Md di Debat Keempat...,NaN,NaN,3,1 rb
9,@resyort,2024-01-11T06:03:14.000Z,Wow mantap nih dukungan terhadap Ganjar Mahfud...,NaN,NaN,NaN,6


In [4]:
df.head()

,userTags,timeStamps,tweets,replies,retweets,likes,views
0,@ShenoWirang,2024-01-10T07:53:20.000Z,"Ketika Pilpres Filipina tahun 2022 ""Bongbong"" ...",11.0,9.0,58,2 rb
1,@detikcom,2024-01-07T16:02:06.000Z,"#DebatPilpres Dalam menutup debat, Anies-Cak I...",8.0,15.0,90,17 rb
2,@P_kenjeran,2024-01-09T03:09:30.000Z,Saya rasa Prabowo pas sekolah lewat jalur khus...,1.0,4.0,10,621
3,@KompasTV,2024-01-12T12:15:00.000Z,Debat Pilpres ketiga antar capres berlangsung ...,1.0,NaN,1,1 rb
4,@kompascom,2024-01-09T22:32:13.000Z,Seorang kepala negara sekaligus kepala pemerin...,108.0,39.0,149,27 rb


In [5]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user', 'time', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag"},
    # fix HTML tokens
    fix_html=True,
    # corpus from which the word statistics are going to be used for word segmentation
    segmenter="twitter",
    # corpus from which the word statistics are going to be used for spell correction
    corrector="twitter",
    # perform word segmentation on hashtags
    unpack_hashtags=True,
    # Unpack contractions (can't -> can not)
    unpack_contractions=True,
    # spell correction for elongated words
    spell_correct_elong=False,
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    # list of dictionaries for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


C:\Users\ZULFIKAR\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...


C:\Users\ZULFIKAR\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading twitter - 1grams ...


In [6]:
def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_zulfikar(text):
    return " ".join(text.split())

def remove_emoji_zulfikar(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('RT'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "")  # don't remove hyphens
    pattern = r"[{}]".format(remove)  # create the pattern
    return re.sub(pattern, "", text)


In [7]:
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text)

def remove_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ")

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)


In [8]:
print(df.columns)


Index(['userTags', 'timeStamps', 'tweets', 'replies', 'retweets', 'likes',
       'views'],
      dtype='object')


In [9]:
# Misalkan Anda ingin mengganti nama kolom 'Tweet' menjadi 'Tweets'
df.rename(columns={'tweets': 'Tweet'}, inplace=True)


In [10]:
print(df.columns)


Index(['userTags', 'timeStamps', 'Tweet', 'replies', 'retweets', 'likes',
       'views'],
      dtype='object')


In [11]:
i = 0
final_string = []

for text in df['Tweet'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = remove_rt(text)
    proc = lower(proc)
    proc = change_stripe(proc)
    proc = remove_emoji_zulfikar(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_zulfikar(proc)
    proc = bersih_data(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc)
    proc = remove_URL_eks(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)


In [12]:
df["step01"] = final_string

In [13]:
df.head(10)

,userTags,timeStamps,Tweet,replies,retweets,likes,views,step01
0,@ShenoWirang,2024-01-10T07:53:20.000Z,"Ketika Pilpres Filipina tahun 2022 ""Bongbong"" ...",11.0,9.0,58,2 rb,ketika pilpres filipina tahun bongbong menol...
1,@detikcom,2024-01-07T16:02:06.000Z,"#DebatPilpres Dalam menutup debat, Anies-Cak I...",8.0,15.0,90,17 rb,dalam menutup debat anies cak imin sebut indon...
2,@P_kenjeran,2024-01-09T03:09:30.000Z,Saya rasa Prabowo pas sekolah lewat jalur khus...,1.0,4.0,10,621,saya rasa prabowo pas sekolah lewat jalur khus...
3,@KompasTV,2024-01-12T12:15:00.000Z,Debat Pilpres ketiga antar capres berlangsung ...,1.0,NaN,1,1 rb,debat pilpres ketiga antar capres berlangsung ...
4,@kompascom,2024-01-09T22:32:13.000Z,Seorang kepala negara sekaligus kepala pemerin...,108.0,39.0,149,27 rb,seorang kepala negara sekaligus kepala pemerin...
5,@NasDem,2024-01-07T13:46:16.000Z,"Calon presiden (capres) nomor urut 1, Anies Ba...",3.0,65.0,213,2 rb,calon presiden capres nomor urut anies baswe...
6,@islah_bahrawi,2024-01-08T07:59:48.000Z,"""Prabowo memiliki hak tanah 220 ribu hektar di...",209.0,877.0,3 rb,148 rb,prabowo memiliki hak tanah ribu hektar di ka...
7,@detikcom,2024-01-09T14:50:00.000Z,Sandiaga singgung soal format debat yang menya...,54.0,11.0,47,45 rb,sandiaga singgung soal format debat yang menya...
8,@tempodotco,2024-01-10T22:04:05.000Z,Hadapi Cak Imin dan Mahfud Md di Debat Keempat...,NaN,NaN,3,1 rb,hadapi cak imin dan mahfud md di debat keempat...
9,@resyort,2024-01-11T06:03:14.000Z,Wow mantap nih dukungan terhadap Ganjar Mahfud...,NaN,NaN,NaN,6,wow mantap nih dukungan terhadap ganjar mahfud...


In [14]:
df_delete = df[df['step01'].str.contains(" ")]

In [15]:
df_new = df[df.isin(df_delete)].dropna()


In [16]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 0 to 19
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   userTags    8 non-null      object 
 1   timeStamps  8 non-null      object 
 2   Tweet       8 non-null      object 
 3   replies     8 non-null      float64
 4   retweets    8 non-null      float64
 5   likes       8 non-null      object 
 6   views       8 non-null      object 
 7   step01      8 non-null      object 
dtypes: float64(2), object(6)
memory usage: 576.0+ bytes


In [17]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ZULFIKAR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
def word_tokenize_wrappper(text):
    return word_tokenize(text)

In [19]:
df_new['tokens'] = df['step01'].apply(word_tokenize_wrappper)

In [20]:
df_new.head(10)

,userTags,timeStamps,Tweet,replies,retweets,likes,views,step01,tokens
0,@ShenoWirang,2024-01-10T07:53:20.000Z,"Ketika Pilpres Filipina tahun 2022 ""Bongbong"" ...",11.0,9.0,58,2 rb,ketika pilpres filipina tahun bongbong menol...,"[ketika, pilpres, filipina, tahun, bongbong, m..."
1,@detikcom,2024-01-07T16:02:06.000Z,"#DebatPilpres Dalam menutup debat, Anies-Cak I...",8.0,15.0,90,17 rb,dalam menutup debat anies cak imin sebut indon...,"[dalam, menutup, debat, anies, cak, imin, sebu..."
2,@P_kenjeran,2024-01-09T03:09:30.000Z,Saya rasa Prabowo pas sekolah lewat jalur khus...,1.0,4.0,10,621,saya rasa prabowo pas sekolah lewat jalur khus...,"[saya, rasa, prabowo, pas, sekolah, lewat, jal..."
4,@kompascom,2024-01-09T22:32:13.000Z,Seorang kepala negara sekaligus kepala pemerin...,108.0,39.0,149,27 rb,seorang kepala negara sekaligus kepala pemerin...,"[seorang, kepala, negara, sekaligus, kepala, p..."
5,@NasDem,2024-01-07T13:46:16.000Z,"Calon presiden (capres) nomor urut 1, Anies Ba...",3.0,65.0,213,2 rb,calon presiden capres nomor urut anies baswe...,"[calon, presiden, capres, nomor, urut, anies, ..."
6,@islah_bahrawi,2024-01-08T07:59:48.000Z,"""Prabowo memiliki hak tanah 220 ribu hektar di...",209.0,877.0,3 rb,148 rb,prabowo memiliki hak tanah ribu hektar di ka...,"[prabowo, memiliki, hak, tanah, ribu, hektar, ..."
7,@detikcom,2024-01-09T14:50:00.000Z,Sandiaga singgung soal format debat yang menya...,54.0,11.0,47,45 rb,sandiaga singgung soal format debat yang menya...,"[sandiaga, singgung, soal, format, debat, yang..."
19,@AirinDatangLagi,2024-01-08T10:31:12.000Z,Yg Kecewa ya Lu \n@jokowi\nKrn Menteri Pertaha...,16.0,35.0,111,1 rb,yg kecewa ya lu jokowi krn menteri pertahanan ...,"[yg, kecewa, ya, lu, jokowi, krn, menteri, per..."


In [21]:
normalized_word = pd.read_csv('kamus_pilpres_normalisasi.csv', sep=",")
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

In [22]:
df_new['final_tokens'] = df_new['tokens'].apply(normalized_term)

In [23]:
i = 0
final_string_tokens = []
for text in df_new['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [24]:
df_new['step02'] = final_string_tokens

In [25]:
df_new["step02"] = final_string_tokens

In [26]:
df_new.head(10)

,userTags,timeStamps,Tweet,replies,retweets,likes,views,step01,tokens,final_tokens,step02
0,@ShenoWirang,2024-01-10T07:53:20.000Z,"Ketika Pilpres Filipina tahun 2022 ""Bongbong"" ...",11.0,9.0,58,2 rb,ketika pilpres filipina tahun bongbong menol...,"[ketika, pilpres, filipina, tahun, bongbong, m...","[ketika, Pemilihan Presiden, filipina, tahun, ...",ketika Pemilihan Presiden filipina tahun bongb...
1,@detikcom,2024-01-07T16:02:06.000Z,"#DebatPilpres Dalam menutup debat, Anies-Cak I...",8.0,15.0,90,17 rb,dalam menutup debat anies cak imin sebut indon...,"[dalam, menutup, debat, anies, cak, imin, sebu...","[dalam, menutup, debat, Anies Baswedan, cak, i...",dalam menutup debat Anies Baswedan cak imin se...
2,@P_kenjeran,2024-01-09T03:09:30.000Z,Saya rasa Prabowo pas sekolah lewat jalur khus...,1.0,4.0,10,621,saya rasa prabowo pas sekolah lewat jalur khus...,"[saya, rasa, prabowo, pas, sekolah, lewat, jal...","[saya, rasa, Prabowo Subianto, pas, sekolah, l...",saya rasa Prabowo Subianto pas sekolah lewat j...
4,@kompascom,2024-01-09T22:32:13.000Z,Seorang kepala negara sekaligus kepala pemerin...,108.0,39.0,149,27 rb,seorang kepala negara sekaligus kepala pemerin...,"[seorang, kepala, negara, sekaligus, kepala, p...","[seorang, kepala, negara, sekaligus, kepala, p...",seorang kepala negara sekaligus kepala pemerin...
5,@NasDem,2024-01-07T13:46:16.000Z,"Calon presiden (capres) nomor urut 1, Anies Ba...",3.0,65.0,213,2 rb,calon presiden capres nomor urut anies baswe...,"[calon, presiden, capres, nomor, urut, anies, ...","[calon, presiden, capres, nomor, urut, Anies B...",calon presiden capres nomor urut Anies Basweda...
6,@islah_bahrawi,2024-01-08T07:59:48.000Z,"""Prabowo memiliki hak tanah 220 ribu hektar di...",209.0,877.0,3 rb,148 rb,prabowo memiliki hak tanah ribu hektar di ka...,"[prabowo, memiliki, hak, tanah, ribu, hektar, ...","[Prabowo Subianto, memiliki, hak, tanah, ribu,...",Prabowo Subianto memiliki hak tanah ribu hekta...
7,@detikcom,2024-01-09T14:50:00.000Z,Sandiaga singgung soal format debat yang menya...,54.0,11.0,47,45 rb,sandiaga singgung soal format debat yang menya...,"[sandiaga, singgung, soal, format, debat, yang...","[sandiaga, singgung, soal, format, debat, yang...",sandiaga singgung soal format debat yang menya...
19,@AirinDatangLagi,2024-01-08T10:31:12.000Z,Yg Kecewa ya Lu \n@jokowi\nKrn Menteri Pertaha...,16.0,35.0,111,1 rb,yg kecewa ya lu jokowi krn menteri pertahanan ...,"[yg, kecewa, ya, lu, jokowi, krn, menteri, per...","[yg, kecewa, ya, lu, Joko Widodo, krn, menteri...",yg kecewa ya lu Joko Widodo krn menteri pertah...


In [27]:
df_new.to_csv('clean_dataset_01.csv', sep=",")
import os
os.system('start "csv" "clean_dataset_01.csv"')


0

## Stemming


In [28]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [29]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stop_words = ['sih', 'nya', 'rt', 'loh', 'lah', 'dd', 'mah', 'nye', 'eh', 'ehh', 'ah', 'yang']
data = factory.get_stop_words()+stop_words
stopwords_sastrawi = factory.create_stop_word_remover()


In [30]:
df_new['step02'] = df_new['step02'].apply(str)

In [31]:
df_new.head()

,userTags,timeStamps,Tweet,replies,retweets,likes,views,step01,tokens,final_tokens,step02
0,@ShenoWirang,2024-01-10T07:53:20.000Z,"Ketika Pilpres Filipina tahun 2022 ""Bongbong"" ...",11.0,9.0,58,2 rb,ketika pilpres filipina tahun bongbong menol...,"[ketika, pilpres, filipina, tahun, bongbong, m...","[ketika, Pemilihan Presiden, filipina, tahun, ...",ketika Pemilihan Presiden filipina tahun bongb...
1,@detikcom,2024-01-07T16:02:06.000Z,"#DebatPilpres Dalam menutup debat, Anies-Cak I...",8.0,15.0,90,17 rb,dalam menutup debat anies cak imin sebut indon...,"[dalam, menutup, debat, anies, cak, imin, sebu...","[dalam, menutup, debat, Anies Baswedan, cak, i...",dalam menutup debat Anies Baswedan cak imin se...
2,@P_kenjeran,2024-01-09T03:09:30.000Z,Saya rasa Prabowo pas sekolah lewat jalur khus...,1.0,4.0,10,621,saya rasa prabowo pas sekolah lewat jalur khus...,"[saya, rasa, prabowo, pas, sekolah, lewat, jal...","[saya, rasa, Prabowo Subianto, pas, sekolah, l...",saya rasa Prabowo Subianto pas sekolah lewat j...
4,@kompascom,2024-01-09T22:32:13.000Z,Seorang kepala negara sekaligus kepala pemerin...,108.0,39.0,149,27 rb,seorang kepala negara sekaligus kepala pemerin...,"[seorang, kepala, negara, sekaligus, kepala, p...","[seorang, kepala, negara, sekaligus, kepala, p...",seorang kepala negara sekaligus kepala pemerin...
5,@NasDem,2024-01-07T13:46:16.000Z,"Calon presiden (capres) nomor urut 1, Anies Ba...",3.0,65.0,213,2 rb,calon presiden capres nomor urut anies baswe...,"[calon, presiden, capres, nomor, urut, anies, ...","[calon, presiden, capres, nomor, urut, Anies B...",calon presiden capres nomor urut Anies Basweda...


In [32]:
i = 0
final_string = []
s = ""
for sentence in df_new["step02"].values:
    filteredSentence = []
    EachReviewText = ""
    st = stopwords_sastrawi.remove(sentence)
    s = (stemmer.stem(st))
    filteredSentence.append(s)
    EachReviewText = ' '.join(filteredSentence)
    final_string.append(EachReviewText)

In [33]:
df_new.loc[:,('ProcessedText')] = final_string

In [34]:
df_new.head()

,userTags,timeStamps,Tweet,replies,retweets,likes,views,step01,tokens,final_tokens,step02,ProcessedText
0,@ShenoWirang,2024-01-10T07:53:20.000Z,"Ketika Pilpres Filipina tahun 2022 ""Bongbong"" ...",11.0,9.0,58,2 rb,ketika pilpres filipina tahun bongbong menol...,"[ketika, pilpres, filipina, tahun, bongbong, m...","[ketika, Pemilihan Presiden, filipina, tahun, ...",ketika Pemilihan Presiden filipina tahun bongb...,pilih presiden filipina tahun bongbong tolak d...
1,@detikcom,2024-01-07T16:02:06.000Z,"#DebatPilpres Dalam menutup debat, Anies-Cak I...",8.0,15.0,90,17 rb,dalam menutup debat anies cak imin sebut indon...,"[dalam, menutup, debat, anies, cak, imin, sebu...","[dalam, menutup, debat, Anies Baswedan, cak, i...",dalam menutup debat Anies Baswedan cak imin se...,tutup debat anies baswedan cak imin sebut indo...
2,@P_kenjeran,2024-01-09T03:09:30.000Z,Saya rasa Prabowo pas sekolah lewat jalur khus...,1.0,4.0,10,621,saya rasa prabowo pas sekolah lewat jalur khus...,"[saya, rasa, prabowo, pas, sekolah, lewat, jal...","[saya, rasa, Prabowo Subianto, pas, sekolah, l...",saya rasa Prabowo Subianto pas sekolah lewat j...,rasa prabowo subianto pas sekolah lewat jalur ...
4,@kompascom,2024-01-09T22:32:13.000Z,Seorang kepala negara sekaligus kepala pemerin...,108.0,39.0,149,27 rb,seorang kepala negara sekaligus kepala pemerin...,"[seorang, kepala, negara, sekaligus, kepala, p...","[seorang, kepala, negara, sekaligus, kepala, p...",seorang kepala negara sekaligus kepala pemerin...,orang kepala negara sekaligus kepala perintah ...
5,@NasDem,2024-01-07T13:46:16.000Z,"Calon presiden (capres) nomor urut 1, Anies Ba...",3.0,65.0,213,2 rb,calon presiden capres nomor urut anies baswe...,"[calon, presiden, capres, nomor, urut, anies, ...","[calon, presiden, capres, nomor, urut, Anies B...",calon presiden capres nomor urut Anies Basweda...,calon presiden capres nomor urut anies basweda...


In [35]:
df_new.to_csv('clean_dataset_02.csv',sep=";")